# LecoSpec Data Munging

In [39]:
# notebooks use their location as their working directory, so
# if we are in a subfolder, move to the main folder.  
# This however can safely be run multiple times
#setwd(M:/lecospec/lecospec)
if(!dir.exists("Functions/")){
    setwd("../")
    if(!dir.exists("Functions")){
        setwd("M:/lecospec/lecospec/")
    }
}
source("Functions/lecospectR.R", echo = FALSE)

In [29]:
6+3+2+4+17+10+9+2

[1] 53

## Notation

Throughout the notebook, variables starting with `img_` are UAV image-based information (data, filepaths, etc).  Similarly, variables beginning with `grd_` related to data collected on the ground.  

Also, some other naming conventions for variables with data transformations:
* `robust` in a variable name refers to data treated by center according to the median and scaling by teh inter-quartile range (a la sklearns RobustScaler)
* `minmax` (and its ilk) are min-max scaled data, i.e. scaled to the interval [0,1] by subtracting the minimum and dividing by the range.
* `standard(ized)` refers to data treated with with the z-score transform by centring using the mean and scaling y the standard deviation (like sklearns StandardScaler)
* `corrected` means that a linear transformation has been applied to account for differences in sensor calibration.
* `raw` refers to having no transformations applied
* `clipped` means that outliers have been clipped to the upper and lower fence values based on the Inter-Quartile Range method. 
* `imputed` means that outliers have been removed and imputed
* `dropped` means that dataframe rows containing outliers have been removed

Example: `img_robust_indices` refers to vegetation indices from the UAV images treated with the robust scaler. 

## Define data locations


In [23]:
# spectral library
grd_base_path <- "./Output/C_001_SC3_Cleaned_SpectralLib.csv"
grd_speclib <- read.csv(grd_base_path, header = TRUE)
#grd_index_path <- ./Data/D_002_SpecLib_Derivs.csv
#grd_indices <- read.csv(grd_index_path)
# this data has some lines that have no labels, so we remove them 
grd_speclib <- grd_speclib[!is.na(grd_speclib$Functional_group1),]
head(grd_speclib)

,X,ScanID,Area,Code_name,Species_name,Functional_group1,Functional_group2,Species_name_Freq,Functional_group1_Freq,Functional_group2_Freq,⋯,Radiometric.Calibration,Units,Latitude,Longitude,Altitude,GPS.Time,Satellites,Calibrated.Reference.Correction.File,Channels,ScanNum
,<int>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<int>,<int>,<int>,⋯,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<chr>,<chr>,<chr>,<int>,<int>
1,1,aleoch_Murph_061,Murphy,aleoch,Alectoria ochroleuca,Lichen,LightTerrestrialMacrolichen,6,453,118,⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
2,2,aleoch_Murph_063,Murphy,aleoch,Alectoria ochroleuca,Lichen,LightTerrestrialMacrolichen,6,453,118,⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
3,3,aleoch_Murph_064,Murphy,aleoch,Alectoria ochroleuca,Lichen,LightTerrestrialMacrolichen,6,453,118,⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
4,4,aleoch_Murph_065,Murphy,aleoch,Alectoria ochroleuca,Lichen,LightTerrestrialMacrolichen,6,453,118,⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
5,5,aleoch_Murph_066,Murphy,aleoch,Alectoria ochroleuca,Lichen,LightTerrestrialMacrolichen,6,453,118,⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
6,6,alnfru_00003,Yukon_Delta,alnfru,Alnus sp.,ShrubDecid,ShrubAlder,82,360,82,⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA


In [4]:
img_base_path <- Data/Ground_Validation/PFT_Image_spectra/PFT_Image_SpectralLib_Clean.csv
img_speclib <- read.csv(img_base_path, header = TRUE)
# currently, not using the old pre-proccessing scheme and just doing it here.
#img_index_path <- Data/D_002_Image_SpecLib_Derivs.csv
#img_speclib <- read.csv(img_base_path)
head(img_speclib)

Okay, there are some metadata columns that should not be there for the next step - lets remove them with `subset`

In [26]:
img_bands <- subset(
    img_speclib, 
    select=-c(
        X,
    	UID,
        ScanNum,
    	sample_name,
    	PFT,
    	FncGrp1
    ))


grd_bands <- subset(
    grd_speclib, 
    select=-c(
        X,
        ScanID,
        Area,
        Code_name,
        Species_name,
        Functional_group1,
        Functional_group2,
        Species_name_Freq,
        Functional_group1_Freq,
        Functional_group2_Freq,
        Genus,
        Version,
        File.Name,
        Instrument,
        Detectors,
        Measurement,
        Date,
        Time,
        Battery.Voltage,
        Averages,
        Integration1,
        Integration2,
        Integration3,
        Dark.Mode,
        Foreoptic,
        Radiometric.Calibration,
        Units,
        Latitude,
        Longitude,
        Altitude,
        GPS.Time,
        Satellites,
        Calibrated.Reference.Correction.File,
        Channels,
        ScanNum
    )
)



Calculate the vegetation indices from the spectral libraries - its easy with lecospectR!

Note that the image-based scpectra are normalized from zero to one, and the ground specctra are on the range zero to one hundred.  

In [27]:
img_indices <- get_vegetation_indices(100*img_bands, NULL)# should have a default of NULL, you know?
grd_indices <- get_vegetation_indices(grd_bands, NULL)

In [28]:
head(img_indices)
head(grd_indices)

,Boochs,Boochs2,CARI,Carter,Carter2,Carter3,Carter4,Carter5,Carter6,CI,⋯,TCARI,TCARIOSAVI,TCARI2,TCARI2OSAVI2,TGI,TVI,Vogelmann,Vogelmann2,Vogelmann3,Vogelmann4
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,0.8972526,0.9977190,40.33337,2.606424,0.1919992,0.1823105,0.4513926,1.687375,10.546899,1.230143,⋯,18.58963,19.90055,29.37852,52.88288,571.6883,2750.790,1.502310,-0.10251539,0.8407581,-0.1132324
2,0.7127460,0.8370667,49.29950,3.283496,0.2205629,0.1740857,0.4311880,2.110946,10.719907,1.103802,⋯,21.02478,22.30213,30.01417,55.92775,590.2966,2753.773,1.467985,-0.10410792,1.1292049,-0.1133074
3,0.6906736,0.8058805,52.33677,3.940434,0.2402597,0.1788333,0.4415758,2.126987,10.846255,1.065731,⋯,21.37050,23.07520,30.62567,59.36034,578.1993,2743.007,1.452730,-0.10563757,1.1222517,-0.1144615
4,0.8406272,0.9835800,38.36841,2.971831,0.1742222,0.1636471,0.4133345,1.643268,10.619240,1.165235,⋯,18.15420,19.32261,28.56205,48.69648,583.6640,2845.341,1.510704,-0.09834182,1.0664627,-0.1087631
5,0.6519263,0.7834586,51.10986,3.397513,0.2156160,0.1586916,0.4100943,2.387455,9.553940,1.056227,⋯,19.87457,20.37338,27.55717,50.37275,538.6860,2710.133,1.494922,-0.12274414,1.1404583,-0.1336822
6,0.8973633,0.9081469,33.54324,2.065685,0.1621322,0.1601592,0.4241145,1.627772,9.855188,1.589680,⋯,16.56562,17.29321,26.13624,43.84734,552.2945,2715.489,1.537855,-0.10372095,0.7253543,-0.1151156


,Boochs,Boochs2,CARI,Carter,Carter2,Carter3,Carter4,Carter5,Carter6,CI,⋯,TCARI,TCARIOSAVI,TCARI2,TCARI2OSAVI2,TGI,TVI,Vogelmann,Vogelmann2,Vogelmann3,Vogelmann4
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,0.1817917,0.1007417,38.29230,4.176483,0.7481783,0.6363165,0.8663692,1.117048,13.0818,1.0250352,⋯,4.638595,17.90381,4.142204,40.63212,52.6595,349.594,1.060078,-0.01470381,0.5351540,-0.01486024
2,0.1779167,0.1020167,28.80530,2.891436,0.7004412,0.6404789,0.8394989,1.101463,11.0986,1.0235607,⋯,4.149470,14.54173,5.683206,43.93186,68.5165,370.886,1.075116,-0.01588072,0.6346180,-0.01612054
3,0.2774667,0.1313083,51.02598,4.620200,0.7542897,0.7048361,0.8855345,1.091549,19.9382,1.0258505,⋯,6.694479,29.33425,7.017488,70.40367,145.4355,525.456,1.052511,-0.01002383,0.5741911,-0.01013679
4,0.3004500,0.1415583,53.56754,4.666115,0.7373047,0.6970615,0.8696484,1.085199,21.6759,1.0229248,⋯,7.194374,30.89612,8.706201,81.47926,169.6470,597.240,1.053829,-0.01009181,0.6619406,-0.01021085
5,0.2610000,0.1341917,45.44606,4.686466,0.7357233,0.6848281,0.8729010,1.083553,17.5634,1.0207577,⋯,5.530025,23.44639,6.766709,63.31335,121.1130,475.050,1.053822,-0.01013896,0.5929445,-0.01025865
6,0.9367000,1.1057000,33.48758,1.427278,0.1320466,0.1048420,0.4051465,1.673969,9.2749,0.9921833,⋯,16.789948,16.99455,23.122057,36.94737,508.4790,2871.108,1.473918,-0.07409511,1.1003322,-0.08200649


This is actually enough to start training models.  We have the vegetation indices, but instead of doing that, let's transform the data and write it to file.  Then we will proceed to creating the model corrections, etc.

In [40]:
img_resampled_bands <- resample_df(100*img_bands, drop_existing=TRUE)
grd_resampled_bands <- resample_df(grd_bands, drop_existing=TRUE)
head(img_resampled_bands)
head(grd_resampled_bands)

Using spline to predict value at new bands...

Beware the spectra are now partially smoothed.

Using spline to predict value at new bands...

Beware the spectra are now partially smoothed.



,X402.593_5nm,X407.593_5nm,X412.593_5nm,X417.593_5nm,X422.593_5nm,X427.593_5nm,X432.593_5nm,X437.593_5nm,X442.593_5nm,X447.593_5nm,⋯,X947.593_5nm,X952.593_5nm,X957.593_5nm,X962.593_5nm,X967.593_5nm,X972.593_5nm,X977.593_5nm,X982.593_5nm,X987.593_5nm,X992.593_5nm
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,3.277507,3.901295,3.922547,3.407865,3.720464,3.820831,3.460818,3.648937,3.674930,3.732257,⋯,60.50174,40.77844,37.61586,50.50814,32.06515,45.66141,56.65918,36.44928,46.68932,55.46659
2,3.277141,3.273556,3.272566,3.291429,3.344372,3.426495,3.524766,3.623183,3.711415,3.788855,⋯,46.86437,46.24061,45.67163,45.30922,45.25675,45.56402,45.97004,46.24739,46.44677,46.58803
3,2.552067,2.695883,2.837947,2.984791,3.139014,3.295136,3.445546,3.584613,3.708338,3.814974,⋯,50.57955,50.53712,50.34267,50.06850,49.77061,49.37533,48.79560,48.07059,47.38716,46.91781
4,3.133775,3.146211,3.020504,2.914408,3.012126,3.203688,3.374192,3.497939,3.571163,3.628934,⋯,56.11057,53.19484,46.40202,42.87151,44.18199,48.42263,49.00493,45.80981,42.90783,44.33673
5,2.888800,2.972499,3.037238,3.090010,3.136556,3.181339,3.225852,3.267638,3.305330,3.337770,⋯,46.06015,44.81342,43.76910,43.05552,42.66213,42.54519,42.60657,42.83058,43.22131,43.67651
6,4.258530,3.863350,3.086016,3.734187,3.394887,2.957398,3.278345,2.936776,3.375559,3.481238,⋯,58.46634,56.04421,47.76229,63.83636,33.53291,57.97247,73.75400,47.06714,44.28437,56.32284


,X402.593_5nm,X407.593_5nm,X412.593_5nm,X417.593_5nm,X422.593_5nm,X427.593_5nm,X432.593_5nm,X437.593_5nm,X442.593_5nm,X447.593_5nm,⋯,X947.593_5nm,X952.593_5nm,X957.593_5nm,X962.593_5nm,X967.593_5nm,X972.593_5nm,X977.593_5nm,X982.593_5nm,X987.593_5nm,X992.593_5nm
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,4.786391,4.478547,4.214955,4.121219,4.261843,4.609276,5.056049,5.609938,6.384817,7.352870,⋯,26.59656,26.71230,26.78679,26.84674,26.92454,27.04128,27.20405,27.38601,27.55660,27.69153
2,4.820802,4.623885,4.461647,4.424839,4.589012,4.933364,5.350729,5.836299,6.480323,7.251967,⋯,21.41355,21.52685,21.62140,21.70847,21.80113,21.90688,22.02912,22.16226,22.29784,22.42359
3,5.849977,5.496151,5.154779,4.986053,5.175844,5.731791,6.499972,7.520754,9.011172,10.881299,⋯,31.84974,31.90839,31.98678,32.05158,32.08263,32.12036,32.19709,32.29237,32.38387,32.46974
4,6.252788,5.817240,5.399310,5.201525,5.424306,6.092052,7.068518,8.364273,10.123249,12.223676,⋯,34.18603,34.21727,34.23974,34.25062,34.25074,34.25434,34.27278,34.30264,34.34254,34.40410
5,5.055413,4.732800,4.457956,4.357744,4.533789,4.989059,5.639766,6.534486,7.839488,9.474391,⋯,29.14092,29.20809,29.27031,29.31446,29.34322,29.40614,29.53721,29.70353,29.86498,29.99403
6,6.056797,5.514715,5.001802,4.665174,4.565842,4.598097,4.511881,4.293868,4.154127,4.142498,⋯,47.87045,47.74562,47.59385,47.48448,47.45815,47.44645,47.39159,47.31926,47.26754,47.26835


In [44]:
img_raw_with_na <- cbind(img_resampled_bands, img_indices)
grd_raw_with_na <- cbind(grd_resampled_bands, grd_indices)


In [46]:
img_raw <- impute_spectra(img_raw_with_na)
grd_raw <- impute_spectra(inf_to_na(grd_raw_with_na))# note also dropping an Inf (liekly div by 0 in veg index)

[1] "Imputing..."
[1] "Imputing..."


Apply the outlier transforms

In [47]:
grd_clipped <- clip_outliers(grd_raw)
grd_imputed <- impute_outliers_and_na(grd_raw)
grd_dropped <- grd_raw[detect_outliers_columnwise(grd_raw),]
img_clipped <- clip_outliers(img_raw)
img_imputed <- impute_outliers_and_na(img_raw)
img_dropped <- img_raw[detect_outliers_columnwise(img_raw),]

[1] "Imputing..."
[1] "Imputing..."


Now the center/scale transforms

In [ ]:
grd_clipped_robust <- columnwise_robust_scale(grd_clipped)
grd_imputed_robust <- columnwise_robust_scale(grd_imputed)
grd_dropped_robust <- columnwise_robust_scale(grd_dropped)
img_clipped_robust <- columnwise_robust_scale(img_clipped)
img_imputed_robust <- columnwise_robust_scale(img_imputed)
img_dropped_robust <- columnwise_robust_scale(img_dropped)

grd_clipped_minmax <- columnwise_min_max_scale(grd_clipped)
grd_imputed_minmax <- columnwise_min_max_scale(grd_imputed)
grd_dropped_minmax <- columnwise_min_max_scale(grd_dropped)
img_clipped_minmax <- columnwise_min_max_scale(img_clipped)
img_imputed_minmax <- columnwise_min_max_scale(img_imputed)
img_dropped_minmax <- columnwise_min_max_scale(img_dropped)

grd_clipped_standard <- standardize_df(grd_clipped)
grd_imputed_standard <- standardize_df(grd_imputed)
grd_dropped_standard <- standardize_df(grd_imputed)
img_clipped_standard <- standardize_df(img_clipped)
img_imputed_standard <- standardize_df(img_imputed)
img_dropped_standard <- standardize_df(img_dropped)

Now, let's save all these data to disk

In [ ]:
if(!dir.exists("Data/gs/")){
    dir.create("Data/gs/")
}

In [ ]:
write.csv(grd_clipped, file="Data/gs/grd_clipped_raw.csv")
write.csv(grd_clipped_minmax, file="Data/gs/grd_clipped_minmax.csv")
write.csv(grd_clipped_robust, file="Data/gs/grd_clipped_robust.csv")
write.csv(grd_clipped_standard, file="Data/gs/grd_clipped_standard.csv")

write.csv(grd_imputed, file="Data/gs/grd_imputed_raw.csv")
write.csv(grd_imputed_minmax, file="Data/gs/grd_imputed_minmax.csv")
write.csv(grd_imputed_robust, file="Data/gs/grd_imputed_robust.csv")
write.csv(grd_imputed_standard, file="Data/gs/grd_imputed_standard.csv")

write.csv(grd_dropped, file="Data/gs/grd_dropped_raw.csv")
write.csv(grd_dropped_minmax, file="Data/gs/grd_dropped_minmax.csv")
write.csv(grd_dropped_robust, file="Data/gs/grd_dropped_robust.csv")
write.csv(grd_dropped_standard, file="Data/gs/grd_dropped_standard.csv")

In [ ]:
write.csv(img_clipped, file="Data/gs/img_clipped_raw.csv")
write.csv(img_clipped_minmax, file="Data/gs/img_clipped_minmax.csv")
write.csv(img_clipped_robust, file="Data/gs/img_clipped_robust.csv")
write.csv(img_clipped_standard, file="Data/gs/img_clipped_standard.csv")

write.csv(img_imputed, file="Data/gs/img_imputed_raw.csv")
write.csv(img_imputed_minmax, file="Data/gs/img_imputed_minmax.csv")
write.csv(img_imputed_robust, file="Data/gs/img_imputed_robust.csv")
write.csv(img_imputed_standard, file="Data/gs/img_imputed_standard.csv")

write.csv(img_dropped, file="Data/gs/img_dropped_raw.csv")
write.csv(img_dropped_minmax, file="Data/gs/img_dropped_minmax.csv")
write.csv(img_dropped_robust, file="Data/gs/img_dropped_robust.csv")
write.csv(img_dropped_standard, file="Data/gs/img_dropped_standard.csv")